In [1]:
import cv2 as cv

import torch
import torchvision
import torch.nn as nn

import numpy as np
import pandas as pd
import PIL
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from evaluation.vlc import VLC

from evaluation.vlc import VLC
from utils.path_utils import Files
from evaluation.simulator import TimingConfig
from evaluation.analysis import Plotter

from neural.dataset import Dataset, numpyDataset
from neural.mlp import MLP
from neural.training import Trainer, MLPTrainer
from neural.train_results import FitResult

%matplotlib qt
pd.options.display.max_columns = 30

torch.set_printoptions(sci_mode=False)

c:\Users\slevylab\AppData\Local\miniforge3\envs\bio_proj\Lib\site-packages\onnxruntime\capi\onnxruntime_validation.py:26: UserWarning: Unsupported Windows version (11). ONNX Runtime supports Windows 10 and above, only.
  warnings.warn(


In [4]:
seed = 42
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

In [ ]:
dataset_X_path = "/Users/guycohen/Desktop/Bio-Project/dataset/dataset_X.npy"
dataset_y_path = "/Users/guycohen/Desktop/Bio-Project/dataset/dataset_y.npy"

# dataset_X = pd.read_csv(dataset_X_path).to_numpy(dtype=np.float32, copy=True)
# dataset_y = pd.read_csv(dataset_y_path).to_numpy(dtype=np.float32)

dataset_X = np.load(dataset_X_path, allow_pickle=True)
dataset_y = np.load(dataset_y_path, allow_pickle=True)


dataset = numpyDataset(dataset_X, dataset_y)


In [5]:
import torch.utils
import torch.utils.data


ds_train, ds_test = torch.utils.data.random_split(dataset, [0.8, 0.2])

In [6]:
# np.set_printoptions(suppress=True)

ds_train.dataset[:10][1]

tensor([[ 5.6719, 11.5062],
        [ 5.8037, 11.7850],
        [ 5.5585, 11.5573],
        [ 5.6195, 11.2552],
        [ 5.8413, 10.4437],
        [ 6.2894, 10.4937],
        [ 6.7957, 10.3788],
        [ 7.1958, 10.4677],
        [ 7.0840, 10.4348],
        [ 7.3099,  9.9405]])

In [7]:
from torch.utils.data import DataLoader

dl_train = DataLoader(ds_train, batch_size=256, shuffle=True)
dl_test = DataLoader(ds_test, batch_size=256, shuffle=True)

In [8]:
hidden_dims = [120, 40, 12, 4, 2]
activaations = ['relu'] * (len(hidden_dims))# + ['softmax']
in_dim = ds_train[0][0].shape[0]
model = MLP(in_dim=in_dim, dims=hidden_dims, nonlins=activaations)


In [9]:
print(model)

MLP(
  (sequence): Sequential(
    (0): Linear(in_features=20, out_features=120, bias=True)
    (1): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=120, out_features=40, bias=True)
    (4): BatchNorm1d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=40, out_features=12, bias=True)
    (7): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Linear(in_features=12, out_features=4, bias=True)
    (10): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): Linear(in_features=4, out_features=2, bias=True)
    (13): Identity()
    (14): Identity()
  )
)


In [10]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=2e-5)

In [11]:
trainer = MLPTrainer(model, loss_fn, optimizer, device=device)

In [12]:
trainer.fit(dl_train=dl_train, dl_test=dl_test, num_epochs=1000, checkpoints='mlpRELU', print_every=5)

--- EPOCH 1/1000 ---


train_batch:   0%|          | 0/200 [00:00<?, ?it/s]

test_batch:   0%|          | 0/50 [00:00<?, ?it/s]


*** Saved checkpoint mlpRELU :: val_loss=44.768

*** Saved checkpoint mlpRELU :: val_loss=43.130

*** Saved checkpoint mlpRELU :: val_loss=41.415

*** Saved checkpoint mlpRELU :: val_loss=40.340

*** Saved checkpoint mlpRELU :: val_loss=39.308
--- EPOCH 6/1000 ---


train_batch:   0%|          | 0/200 [00:00<?, ?it/s]

...

test_batch:   0%|          | 0/50 [00:00<?, ?it/s]

--- EPOCH 141/1000 ---


train_batch:   0%|          | 0/200 [00:00<?, ?it/s]

test_batch:   0%|          | 0/50 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
model.save

# Dataset Creation

In [ ]:
timing_config = TimingConfig.load_json("/Users/guycohen/Desktop/Bio-Project/time_config.json")

log_path = "/Users/guycohen/Desktop/Bio-Project/eval/bboxes2.csv"


pltr = Plotter(log_path, timing_config)
data = pltr._data.copy()

In [ ]:
df = pd.DataFrame(index=range(8), columns=['x', 'y'])
df.iloc[0]  = [1, 0]
df

In [ ]:
data.iloc[[0,-1]]


In [ ]:
# data.ag
data.rolling(window=10, method='table').flatten()

In [ ]:
np.arange(40).reshape(8,5)[[2,3]].flatten()

In [ ]:
data2 = data.dropna(inplace=False, ignore_index=False, axis=0)
mask = data.isna().any(axis=1)
data['']

In [2]:
# make sure that 0 is first at frames_for_pred
def create_dataset(log_path: str, frames_for_pred=[0,-3,-15], pred_frame=7, save_path:str=None) -> Dataset:
    data = pd.read_csv(log_path)
    start_idx = abs(min(frames_for_pred)) + 1
    X_mask = np.asanyarray(frames_for_pred)
    y_mask = np.asanyarray([pred_frame])

    data = Plotter.concat(data, Plotter.calc_centers(data, "wrm"))
    
    wrm_centers = data[['wrm_center_x', 'wrm_center_y']].to_numpy(dtype=np.float64)
    wrm_boxes = data[['wrm_x', 'wrm_y', 'wrm_w', 'wrm_h']].to_numpy(dtype=np.float64)
    
    # Create columns for X and y
    cols = ['wrm_x', 'wrm_y', 'wrm_w', 'wrm_h']
    y_cols = ['wrm_center_x', 'wrm_center_y']
    X_cols = []
    for i in frames_for_pred:
        X_cols += [col+str(i) for col in cols]
    
    # Create X and y
    X = pd.DataFrame(index=data.index, columns=X_cols)
    y = pd.DataFrame(index=data.index, columns=y_cols)
    for i in range(start_idx, len(data)-pred_frame-1):
        X.iloc[i] = wrm_boxes[i + X_mask].reshape(1, -1)
        y.iloc[i] = wrm_centers[i + y_mask].reshape(1, -1)

    # Drop rows with NaN values
    na_mask = np.ma.mask_or(X.isna().any(axis=1), y.isna().any(axis=1))
    X = X.loc[~na_mask]
    y = y.loc[~na_mask]
    
    X = X.to_numpy(dtype=np.float32, copy=True)
    y = y.to_numpy(dtype=np.float32, copy=True)

    # make X and y coordinates relative to the prediction frame
    x_cord_mask = np.arange(X.shape[1]) % 4 == 0
    y_cord_mask = np.arange(X.shape[1]) % 4 == 1
    
    x_cords = X[:, 0].reshape(-1, 1)
    y_cords = X[:, 1].reshape(-1, 1)

    y[:, [0]] -= x_cords
    y[:, [1]] -= y_cords
    X[:, x_cord_mask] -= x_cords#
    X[:, y_cord_mask] -= y_cords#.reshape(-1, 1)
    
    if save_path is not None:
        np.save(save_path + "dataset_X_3.npy", X, allow_pickle=True)
        np.save(save_path + "dataset_y_3.npy", y, allow_pickle=True)
        # X.to_csv(save_path + "dataset_X.csv", index=False)
        # y.to_csv(save_path + "dataset_y.csv", index=False)

    return numpyDataset(X, y)

In [3]:
# log_path = "/Users/guycohen/Desktop/Bio-Project/eval/bboxes2.csv"
log_path = f"D:/Guy_Gilad/Exp1_GuyGilad/logs_yolo/bboxes.csv"
pred_frame=7
frames_for_pred=[0,-3,-15,-18,-30]
dataset = create_dataset(log_path, save_path="D:\Guy_Gilad\Bio-Proj\evaluation", frames_for_pred=frames_for_pred, pred_frame=pred_frame)

<>:5: SyntaxWarning: invalid escape sequence '\G'
<>:5: SyntaxWarning: invalid escape sequence '\G'
C:\Users\slevylab\AppData\Local\Temp\ipykernel_2476\3900827647.py:5: SyntaxWarning: invalid escape sequence '\G'
  dataset = create_dataset(log_path, save_path="D:\Guy_Gilad\Bio-Proj\evaluation", frames_for_pred=frames_for_pred, pred_frame=pred_frame)


In [ ]:
X, y = ds[:]
df = pd.DataFrame({"dx": y[:,0], "dy": y[:,1]})

In [ ]:
df.hist()